In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd 

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [2]:
import pandas as pd 

batch_size = 32
img_height = 48
img_width = 48

data_dir = "train"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6176 files belonging to 2 classes.
Using 4941 files for training.
Found 6176 files belonging to 2 classes.
Using 1235 files for validation.


In [3]:
print(train_ds.class_names)


['happy', 'sadness']


In [13]:
from tensorflow.keras.applications.vgg16 import VGG16
from keras.optimizers import Adam

batch_size = 32
img_height = 48
img_width = 48

base_model = VGG16(input_shape = (img_height, img_width, 3),
                  include_top=False,    
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False 

    
##### FULLY CONNECTED LAYER #####
# creamos una pequeña red al final del modelo para entrenarlo para nuestros datos

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x) 

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x) 

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Aquí creamos un modelo concatenado, el ya existente y nuestras nuevas capas
model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = "Adam", loss = 'binary_crossentropy',metrics = ['acc'])

In [14]:
vgghist = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 8,
                    verbose=1)

Epoch 1/8
155/155 [==============================] - 23s 146ms/step - loss: 2.3421 - acc: 0.6021 - val_loss: 0.8447 - val_acc: 0.6534
Epoch 2/8
155/155 [==============================] - 22s 145ms/step - loss: 0.8144 - acc: 0.6659 - val_loss: 0.6284 - val_acc: 0.6858
Epoch 3/8
155/155 [==============================] - 22s 144ms/step - loss: 0.6244 - acc: 0.7110 - val_loss: 0.6055 - val_acc: 0.6964
Epoch 4/8
155/155 [==============================] - 23s 148ms/step - loss: 0.5499 - acc: 0.7314 - val_loss: 0.5964 - val_acc: 0.7093
Epoch 5/8
155/155 [==============================] - 24s 154ms/step - loss: 0.4948 - acc: 0.7543 - val_loss: 0.5884 - val_acc: 0.7158
Epoch 6/8
155/155 [==============================] - 25s 161ms/step - loss: 0.4704 - acc: 0.7713 - val_loss: 0.5806 - val_acc: 0.7287
Epoch 7/8
155/155 [==============================] - 24s 155ms/step - loss: 0.4402 - acc: 0.7934 - val_loss: 0.5979 - val_acc: 0.7117
Epoch 8/8
155/155 [==============================] - 24s 153ms

In [15]:
train_ds

<BatchDataset shapes: ((None, 48, 48, 3), (None,)), types: (tf.float32, tf.int32)>

In [27]:
pd.read_csv("test_set.csv")["path"]

0       test/18341.jpg
1       test/13176.jpg
2       test/23945.jpg
3       test/15968.jpg
4       test/18382.jpg
             ...      
4112    test/08966.jpg
4113    test/12111.jpg
4114    test/16629.jpg
4115    test/24322.jpg
4116    test/23412.jpg
Name: path, Length: 4117, dtype: object

In [30]:
list_dir = list(pd.read_csv("test_set.csv")["path"])

images = [tf.keras.preprocessing.image.load_img(filename, target_size=(img_height, img_width)) for filename in list_dir]
test_ds = np.array([tf.keras.preprocessing.image.img_to_array(img) for img in images])
test_ds = tf.data.Dataset.from_tensors(test_ds)

predictions = model.predict(test_ds)
results = np.array([np.max(predictions[i]) for i in range(len(predictions))])

sample = pd.read_csv('test_set.csv')
id_col = sample['id_img']

submission = pd.DataFrame({'id_img':id_col, 'label':results}, index=range(len(id_col)))
submission.label = submission.label.apply(lambda x: 1 if x >= 0.5 else 0)
submission.label = submission.label.apply(lambda x: 'happy' if x == 0 else 'sadness')

submission.head()

,id_img,label
0,18341,happy
1,13176,happy
2,23945,sadness
3,15968,happy
4,18382,happy


In [33]:
# sample
sample = pd.read_csv("sample_submission.csv")
sample

,id_img,label
0,18341,sadness
1,13176,sadness
2,23945,happy
3,15968,happy
4,18382,happy
...,...,...
4112,8966,sadness
4113,12111,sadness
4114,16629,sadness
4115,24322,sadness


In [34]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id_img.all() == sample.id_img.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")

In [35]:
chequeator(submission)

You're ready to submit!


NameError: name 'urllib' is not defined